In [8]:
import torch
from ultralytics import YOLO
# Model
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model = YOLO("yolov8n.pt")

In [9]:
def get_prediction_batch(model, images):
    pred = model(images)
    return pred

In [10]:
def yolo_to_voc(x_center, y_center, width, height, img_width, img_height):
    x_center_abs = x_center * img_width
    y_center_abs = y_center * img_height
    width_abs = width * img_width
    height_abs = height * img_height
    
    x_min = int(x_center_abs - width_abs / 2)
    y_min = int(y_center_abs - height_abs / 2)
    x_max = int(x_center_abs + width_abs / 2)
    y_max = int(y_center_abs + height_abs / 2)
    
    return x_min, y_min, x_max, y_max

In [11]:
from shapely.geometry import Polygon

def get_intersection_ratio(polygon, bbox):
    intersection = polygon.intersection(bbox)
    union = polygon.union(bbox)
    # Calculate the intersection ratio (area of intersection / area of the original polygon)
    iou = intersection.area / union.area
    return iou


In [12]:
parking_spots = [
    [(1590,878), (1761, 901), (1756, 1050), (1563, 1036)],
[(1467, 757), (1593, 828), (1559, 996), (1410, 896)],
[(1353, 684), (1459, 736), (1405, 885), (1287, 807)],
[(1243, 639), (1341, 654), (1280, 795), (1183, 732)],
[(1153, 588), (1235, 617), (1166, 733), (1084, 677)],
[(1107, 537), (1158, 577), (1085, 668), (1020, 616)],
[(1016, 493), (1089, 520), (1016, 609), (961, 563)],
[(963, 462), (1031, 498), (944, 564), (895, 534)],
[(909, 428), (972, 453), (903, 517), (852, 496)],
[(878, 401), (920, 416), (849, 485), (803, 437)]
]

In [14]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pybboxes

def iterate_folder(path, results_path):
    images = os.listdir(path)
    for image in images:
        if image[-4:] == '.jpg':
            print(image)
            img_path = os.path.join(path, image)
            img = cv2.imread(img_path)

            pred = get_prediction_batch(model, img)
            
            labels, cord_thres = pred[0].boxes.data[:, -1].numpy().astype(int), pred[0].boxes.data[:, :-1].numpy()

            np.random.seed(42)  # For reproducibility
            iou_ratios = []
            colors = np.random.randint(0, 255, size=(len(pred), 3), dtype='uint8')
            for j in range(len(parking_spots)):
                spot = parking_spots[j]
                polygon = Polygon(spot)
                for i, (coords, label) in enumerate(zip(cord_thres, labels)):
                    if label == 2:
                        conf = coords[4]
                        coords = coords.astype(int)
                        x1, y1, x2, y2 = coords[0], coords[1], coords[2], coords[3] #yolo_to_voc(coords[0], coords[1], coords[2], coords[3], img.shape[1], img.shape[0])
                        
                        voc_bbox = (x1, y1, x2, y2)
                        bbox_points = [(voc_bbox[0], voc_bbox[1]), (voc_bbox[0], voc_bbox[3]), (voc_bbox[2], voc_bbox[3]), (voc_bbox[2], voc_bbox[1])]
                        bbox = Polygon(bbox_points)
                        ratio = get_intersection_ratio(polygon, bbox)
                        pts = np.array([spot],
                                    np.int32)
                        pts = pts.reshape((-1, 1, 2))

                        if ratio > 0.3:
                            iou_ratios.append((ratio, j, i))

            sorted_list = sorted(iou_ratios, key=lambda x: x[0])
            parking_occupied = [0 for i in range(0, 10)]
            car_used = [0 for i in range(len(cord_thres))]
            for tpl in sorted_list:
                if parking_occupied[tpl[1]] == 0 and car_used[tpl[2]] == 0:
                    parking_occupied[tpl[1]] = 1
                    car_used[tpl[2]] = 1
            txt_path = os.path.join(path, image[:-4] + '_query.txt')
            file1 = open(txt_path, 'r')
            Lines = file1.readlines()
            query = []
            all_true = True
            results = []
            for line in Lines[1:]:
                line = line.strip().split(" ")
                results.append( (int(line[0]), parking_occupied[int(line[0])-1]) )
            file_path = image[:-4] + "_predicted.txt"
            with open(os.path.join(results_path, file_path), 'a') as file:
                file.write(str(len(results)) + '\n')
                for line in results:
                    to_write = str(line[0]) + " " + str(line[1]) 
                    file.write(to_write + '\n')
        

            


test_path = "./test/Task1"

results_path = "./submission_files/407_Serafim_Alex/Task1"
if not os.path.exists(results_path):
    os.makedirs(results_path)

iterate_folder(test_path, results_path)

01_1.jpg

0: 352x640 24 cars, 1 truck, 229.9ms
Speed: 8.2ms preprocess, 229.9ms inference, 8.3ms postprocess per image at shape (1, 3, 352, 640)
01_2.jpg

0: 352x640 3 persons, 21 cars, 1 truck, 161.1ms
Speed: 8.2ms preprocess, 161.1ms inference, 8.2ms postprocess per image at shape (1, 3, 352, 640)
01_3.jpg

0: 352x640 2 persons, 24 cars, 130.5ms
Speed: 0.0ms preprocess, 130.5ms inference, 8.2ms postprocess per image at shape (1, 3, 352, 640)
02_1.jpg

0: 352x640 2 persons, 18 cars, 131.2ms
Speed: 0.0ms preprocess, 131.2ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)
02_2.jpg

0: 352x640 1 person, 12 cars, 146.6ms
Speed: 0.0ms preprocess, 146.6ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)
02_3.jpg

0: 352x640 2 persons, 16 cars, 1 truck, 1 traffic light, 132.6ms
Speed: 9.1ms preprocess, 132.6ms inference, 8.1ms postprocess per image at shape (1, 3, 352, 640)
03_1.jpg

0: 352x640 1 person, 25 cars, 1 traffic light, 130.4ms
Speed: 8.2ms preproce